# Simulação de rag via ollama 

ref.: 
- https://medium.com/@imabhi1216/implementing-rag-using-langchain-and-ollama-93bdf4a9027c
- release de resultados - petrobras: https://api.mziq.com/mzfilemanager/v2/d/25fdf098-34f5-4608-b7fa-17d60b2de47d/ebaa356a-b332-7a81-b61d-4a3ebb107157?origin=1

In [1]:
from langchain_community.document_loaders import PDFPlumberLoader

from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_ollama import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

## Lendo o pdf

In [2]:
loader = PDFPlumberLoader("./resultados_petrobras.pdf")
docs = loader.load()
# Check the number of pages
print("Número de paginas:",len(docs))

Número de paginas: 18


In [3]:
# Load the random page content
docs[2].page_content

'Nossos Resultados Operacionais\nDestaques - 2T24\nNo 2T24 a produção média de óleo, LGN e gás natural da\nProdução Total Própria\nPetrobras alcançou 2.699 Mboed. Na comparação com a\nmilhões boed\nprodução do mesmo período do ano anterior (2T23),\ntivemos um aumento de 2,4%. Dentre os principais\n2,64 2,78 2,70\nfatores para essa variação podemos destacar o ramp-up\ndos FPSOs Almirante Barroso, P-71, Anna Nery, Anita 0,04 0,03 0,03\nGaribaldi e Sepetiba, além da entrada em produção de 12 0,50 0,51 0,51\nnovos poços de projetos complementares, sendo 8 na\nBacia de Campos e 4 na Bacia de Santos.\n2,10 2,24 2,16\nEm comparação com 1T24, a produção foi 2,8% inferior\ndevido, principalmente, ao maior volume de perdas por\nparadas para manutenções, dentro do previsto no Plano\nEstratégico 2024-2028+ (PE 2024-2028+), e ao declínio\n2T23 1T24 2T24\nnatural de campos maduros. Esses fatores foram\nparcialmente compensados pela entrada de novos poços Óleo Gás Óleo e Gás - exterior\nde projetos c

## Criação de chunks

In [4]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

C:\Users\leona\AppData\Local\Temp\ipykernel_33524\2474015895.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\leona\AppData\Local\Temp\ipykernel_33524\2474015895.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\leona\Documents\GitHub\ollama_tests\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: I

In [5]:
print("Número de chunks criados: ", len(documents))
for i in range(2):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Número de chunks criados:  32

CHUNK : 1
Relatório de
Produção e Vendas
2T24
—


CHUNK : 2
Sumário
Destaques - 2T24 3
Nossos Resultados Operacionais 4
Exploração & Produção 4
Refino, Transporte e Comercialização 6
Gás e Energias de Baixo Carbono 9
Emissões Atmosféricas 10
Anexos 12
ANEXO I - VOLUME DE VENDAS CONSOLIDADO 12
ANEXO II - EXPORTAÇÃO E IMPORTAÇÃO LÍQUIDA 13
ANEXO III - EXPORTAÇÕES DE PETRÓLEO 13
ANEXO IV – EXPORTAÇÕES DE DERIVADOS 14
Glossário 15
AVISO
Este relatório pode conter previsões acerca de eventos futuros. Tais previsões refletem apenas expectativas dos
administradores da companhia sobre condições futuras da economia, além do setor de atuação, do desempenho
e dos resultados financeiros da companhia, dentre outros. Os termos “antecipa", "acredita", "espera", "prevê",
"pretende", "planeja", "projeta", "objetiva", "deverá", bem como outros termos similares, visam a identificar tais
previsões, as quais, evidentemente, envolvem riscos e incertezas previstos ou não pela c

## Criando os embeddings

In [6]:
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)

C:\Users\leona\AppData\Local\Temp\ipykernel_33524\3660788537.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


## Recuperando informações dos embeddings

In [7]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("Quais são os resultados operacionais?")
len(retrieved_docs)

3

In [8]:
retrieved_docs[0].metadata['page'], retrieved_docs[0].page_content

(4,
 '“O FPSO Maria Quitéria está em navegação para\no Brasil e deve chegar à locação nas próximas\nsemanas. A unidade tem início de operação\nprevisto para o último trimestre de 2024,\nantecipando assim o cronograma do PE 2024-\n2028+, que era de entrada em operação em 2025. O Integrado Parque das Baleias é um projeto\n100% Petrobras, que faz parte do Programa de\nRenovação da Bacia de Campos, e irá contribuir\npara o aumento da produção, redução do custo\nde extração e das emissões”. Magda Chambriard, Presidente da Petrobras\nPETROBRAS | Relatório de Produção e Vendas | 2T24 5\n')

In [9]:
retrieved_docs[1].metadata['page'], retrieved_docs[1].page_content

(16,
 'Glossário\nP\nProdução total: Produção de óleo, LGN e gás natural (considera o volume de gás natural reinjetado e não\ncomercializado)\nProdução total comercial: Produção de óleo, LGN e gás natural comercial (desconta o volume de gás natural\nreinjetado e não comercializado). Produção total operada: Produção de um campo de gás ou petróleo, incluindo a participação da Petrobras\ne a participação dos parceiros. R\nRegaseificação de GNL: volume operacional de GNL que foi regaseificado e disponibilizado pela Petrobras\npara o mercado na saída dos terminais de GNL, convertido para o PCS de referência de 9400 kcal/m³. Os\nvolumes que foram transferidos dos navios metaneiros para os navios regaseificadores mas ainda não\nforam regaseificados não compõem esta medida. Refino, Transporte e Comercialização (RTC): O segmento abrange as atividades de refino, logística,\ntransporte, aquisição e exportação de petróleo bruto, bem como negociação de derivados de petróleo no\nBrasil e no exterior

In [10]:
retrieved_docs[2].metadata['page'], retrieved_docs[2].page_content

(13, 'PETROBRAS | Relatório de Produção e Vendas | 2T24 14\n')

## Conversando com o documento

In [12]:
llm = Ollama(model="llama3.2")

In [16]:
prompt = """
Use os seguintes elementos de contexto para responder à pergunta no final.
Se você não souber a resposta, apenas diga "Eu não sei", mas não invente uma resposta por conta própria.
Mantenha a resposta clara e limitada a 3 ou 4 frases.
Contexto: {context}

Pergunta: {question}

Resposta útil:
"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

print(qa("Como foi a produção de óleo no pré-sal?")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use os seguintes elementos de contexto para responder à pergunta no final.
Se você não souber a resposta, apenas diga "Eu não sei", mas não invente uma resposta por conta própria.
Mantenha a resposta clara e limitada a 3 ou 4 frases.
Contexto: Context:
content:Glossário
P
Produção total: Produção de óleo, LGN e gás natural (considera o volume de gás natural reinjetado e não
comercializado)
Produção total comercial: Produção de óleo, LGN e gás natural comercial (desconta o volume de gás natural
reinjetado e não comercializado). Produção total operada: Produção de um campo de gás ou petróleo, incluindo a participação da Petrobras
e a participação dos parceiros. R
Regaseificação de GNL: volume operacional de GNL que foi regaseificado e disponibilizado pela Petrobras
para o mercado na saída dos terminais de GNL, convertido para o PCS de referência de 9400 kcal/m³. Os
volumes que foram transfe

In [21]:
print(qa("""
Como foi volume de vendas de derivados no 2T24, 
principalmente em função do aumento de vendas de diesel e de GLP?
""")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use os seguintes elementos de contexto para responder à pergunta no final.
Se você não souber a resposta, apenas diga "Eu não sei", mas não invente uma resposta por conta própria.
Mantenha a resposta clara e limitada a 3 ou 4 frases.
Contexto: Context:
content:maior que no 1T24, favorecendo a produção de derivados de maior valor
agregado e a diminuição de emissões. As vendas de diesel S-10 no 2T24 representaram 63,9% das vendas totais de óleo
diesel pela Petrobras, estabelecendo um novo recorde trimestral. O diesel S-10
assume importância cada vez maior no portfólio de produtos da empresa,
apresentando baixo teor de enxofre e melhores resultados ambientais. PETROBRAS | Relatório de Produção e Vendas | 2T24 3

source:./resultados_petrobras.pdf

Context:
content:Nossos Resultados Operacionais
Vendas
O volume de vendas de derivados no 2T24 aumentou 3,2% em relação ao 1T24, principalmente em 